In [ ]:
!pip install -q gradio rake-nltk PyPDF2 pymupdf4llm nltk groq langchain-community chromadb langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.4 MB/s eta 0

In [ ]:
import gradio as gr
import pandas as pd
import pymupdf4llm
import numpy as np
import re
from rake_nltk import Rake
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk

nltk.download('stopwords')
nltk.download('punkt')

nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
prompt_cv="""

You are a highly intelligent assistant tasked with analyzing CVs and creating concise, structured summaries optimized for comparing with job descriptions.

Your goal is to extract and organize key details into specific categories. Structure your response as follows:

1. **Professional Summary**: Provide a Five-sentences max overview of the candidate’s career focus, expertise, and achievements.
2. **Key Skills and Expertise**: Enumerate the candidate's main skills, including technical skills (e.g., programming languages, tools,frameworks, services) and non-technical skills (e.g., leadership, communication).
3. **Work Experience**:
   - For each role, include:
     - Job Title.
     - Organization Name.
     - Duration of Employment.
     - Key responsibilities and accomplishments.
4. **Technologies and Tools**: List every software, programming languages, frameworks,API service, and tools the candidate has experience with.
5. **Education**: Summarize degrees, fields of study, and institutions attended.
6. **Certifications and Training**: Highlight any certifications, courses, or training programs completed.
7. **Languages**: Mention languages the candidate knows and their proficiency levels.
8. **Projects**: Include significant projects or achievements relevant to the candidate's profile.
9. **Keywords**: Extract important keywords that characterize the candidate’s expertise.

Focus on clarity and precision in each section to ensure a well-structured summary. Here's the CV content:

{content}

Respond in the requested structured format.

"""

In [ ]:
GROQ_API_KEY="gsk_uas1v..."

from groq import Groq

def Summarize_job_text(content,prompt):
    """
    Summarize a table using an LLM.
    """
    client = Groq(api_key=GROQ_API_KEY)

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "you are a helpful assistant."
            },
            {
                "role": "user",
                "content": prompt.format(content=content),
            }
        ],
        model="llama3-8b-8192",
        temperature=0,
        max_tokens=2048,
        top_p=1,
    )


    return chat_completion.choices[0].message.content

In [ ]:
import uuid
from langchain.schema.document import Document
import uuid
import pandas as pd
from langchain.vectorstores import Chroma
from langchain.embeddings import JinaEmbeddings

In [ ]:
#unzip the db
!unzip ./job_chroma_db.zip

Archive:  ./job_chroma_db.zip
   creating: content/job_chroma_db/
  inflating: content/job_chroma_db/chroma.sqlite3  
   creating: content/job_chroma_db/20bd1a1b-1584-482f-8742-5ee07f27d823/
  inflating: content/job_chroma_db/20bd1a1b-1584-482f-8742-5ee07f27d823/link_lists.bin  
  inflating: content/job_chroma_db/20bd1a1b-1584-482f-8742-5ee07f27d823/header.bin  
  inflating: content/job_chroma_db/20bd1a1b-1584-482f-8742-5ee07f27d823/index_metadata.pickle  
  inflating: content/job_chroma_db/20bd1a1b-1584-482f-8742-5ee07f27d823/length.bin  
  inflating: content/job_chroma_db/20bd1a1b-1584-482f-8742-5ee07f27d823/data_level0.bin  


In [ ]:
embedding_model = JinaEmbeddings(
    jina_api_key="jina_71a2d1c8cf884d6593a35a6a5d89ad2fp2BEEYkIHuIpXMXv7PIXHfFOk5vU",
    model_name="jina-embeddings-v3"
)

In [ ]:
#load the vectorstore
vectorstore = Chroma(
    persist_directory="./content/job_chroma_db",
    embedding_function=embedding_model
)

<ipython-input-11-7912392b0fd6>:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [ ]:
cv_markdown = pymupdf4llm.to_markdown("/content/Ismail-Oubah-EnglishCV.pdf", page_chunks=True)
cv_text = " ".join([chunk['text'] for chunk in cv_markdown])

# 2. Generate structured summary
cv_summary = Summarize_job_text(cv_text, prompt_cv)


Processing /content/Ismail-Oubah-EnglishCV.pdf...
[                                        ] (0/1)========================================[========================================] (1/1)]


In [ ]:
results = vectorstore.similarity_search_with_score(cv_summary, k=3)

In [ ]:
document=results[0][0]

In [ ]:
document.page_content

In [ ]:
result = vectorstore._collection.get(include=['documents'])

In [ ]:
def process_cv_and_recommend(cv_file, interests):
    """End-to-end processing pipeline with proper score conversion"""
    try:
        cv_markdown = pymupdf4llm.to_markdown(cv_file.name, page_chunks=True)
        cv_text = " ".join([chunk['text'] for chunk in cv_markdown])

        cv_summary = Summarize_job_text(cv_text, prompt_cv)

        # 3. Semantic search (get raw scores)
        results = vectorstore.similarity_search_with_score(cv_summary, k=50)

        # 4. Convert to percentages
        processed = []
        for doc, score in results:
            # Convert cosine distance to similarity percentage
            embedding_score = (1 - score) * 100

            processed.append({
                "Document": doc,
                "Embedding Score": embedding_score,
                "Raw Score": score
            })

        # 5. Calculate interest similarity
        job_titles = [p["Document"].metadata["job_title"] for p in processed]
        interest_scores = calculate_interest_similarity(interests, job_titles) * 100

        # 6. Combine scores
        for i, p in enumerate(processed):
            p["Interest Score"] = interest_scores[i]
            p["Combined Score"] = (0.7 * p["Embedding Score"]) + (0.3 * p["Interest Score"])

        # 7. Sort and format results
        processed.sort(key=lambda x: x["Combined Score"], reverse=True)

        # Build final dataframe
        results_df = pd.DataFrame([{
            "Title": p["Document"].metadata.get("job_title", "N/A"),
            "Company": p["Document"].metadata.get("company", "N/A"),
            "Match %": f"{p['Combined Score']:.1f}%",
            "Skills Match": f"{p['Embedding Score']:.1f}%",
            "Interest Match": f"{p['Interest Score']:.1f}%",
            "Link": p["Document"].metadata.get("job_link", "#"),
            "summary": p['Document'].page_content
        } for p in processed[:50]])  # Show top 20

        return results_df

    except Exception as e:
        return pd.DataFrame({"Error": [str(e)]})

def calculate_interest_similarity(interests, job_titles):
    """Calculate TF-IDF similarity between interests and job titles"""
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(job_titles + [interests])
    return cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])[0]





In [ ]:
# Gradio Interface
with gr.Blocks(title="Job Matcher", theme=gr.themes.Soft()) as app:
    gr.Markdown("## 🔍 AI-Powered Job Matching Engine")

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 📤 Your Profile")
            cv_upload = gr.File(label="Upload your CV (PDF)", file_types=[".pdf"])
            interests_input = gr.Textbox(
                label="Career Interests (comma-separated)",
                placeholder="e.g.: Machine Learning, Data Analysis, Cloud Computing"
            )
            submit_btn = gr.Button("Find Best Matches", variant="primary")

        with gr.Column(scale=2):
            gr.Markdown("### 🏆 Top Recommendations")
            results_table = gr.Dataframe(
                headers=["Title", "Company", "Match %", "Skills Match", "Interest Match", "Link","summary"],
                datatype=["str", "str", "str", "str", "str", "str","str"],
                interactive=False,
                wrap=True
            )

    submit_btn.click(
        fn=process_cv_and_recommend,
        inputs=[cv_upload, interests_input],
        outputs=results_table
    )

    if __name__ == "__main__":
       app.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c59343ef59baf9e1a8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
